In [1]:
import numpy as np
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader


# Initial Tutorial

In [2]:
a = np.ones((2,2))
b = torch.from_numpy(a)
print(b)

tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64)


In [3]:
x = torch.ones(2,2,requires_grad=True)
y = x + 2
print(y.grad_fn)
z = y*y*3
out=z.mean()
print(z, out)

a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)
False
True


In [12]:
#out.backward()
print(x.requires_grad)
print(y.requires_grad)
print(z.requires_grad)


True
True
True


## JACOBIAN

In [37]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

v = torch.tensor([0, 0, 0], dtype=torch.float)
y.backward(v)

tensor([-152.4110, -742.4994, 1345.1473], grad_fn=<MulBackward0>)


# First NN


In [76]:
class Net_own(nn.Module):
    def __init__(self):
        super(Net_own,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,(2,2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x , (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [77]:
net = Net_own()
input = torch.randn(1, 1, 32, 32)
out = net(input)

# Train our model

In [72]:
target = torch.randn(10)
target = target.view(1, -1)
print(target)
criterion = nn.MSELoss()
loss = criterion(out,target)

tensor([[ 0.5233,  0.2993, -0.7794,  1.2261, -1.1652, -2.8852, -0.0426,  0.2344,
          0.8114, -0.1455]])


In [79]:
optimizer = optim.SGD(net.parameters(),lr=0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

# First Word Embedding

In [81]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [82]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [86]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2]) 
            for i in range(len(test_sentence) - 2)]

In [94]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [120]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
class NGramLanguageModeler(nn.Module):
    def __init__(self,vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, inputs):
        #print("INPUTS", inputs)
        embeds = self.embeddings(inputs).view((1, -1))
        print("EMBEDINGS", embeds.shape)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [121]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [122]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses) 

EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS 

EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS 

EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS 

EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS torch.Size([1, 20])
EMBEDINGS 

In [130]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 10
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [131]:
word_to_ix['We']

29

In [160]:
class CBOW(nn.Module):

    def __init__(self,vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = nn.Linear(context_size * 2 * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        inputs = self.embedding(inputs).view((1,-1))
        inputs = F.relu(self.linear1(inputs))
        return self.linear2(inputs)

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)




In [161]:
losses = []
loss_function = nn.CrossEntropyLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [162]:
print(model)

CBOW(
  (embedding): Embedding(49, 10)
  (linear1): Linear(in_features=40, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=49, bias=True)
)


In [166]:
for epoch in range(10):
    total_loss = 0
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses) 

[230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.01091480255127, 230.13825249671936, 228.79691624641418, 227.46571278572083, 226.14464378356934, 224.83260345458984, 223.52915263175964, 222.23388838768005, 220.9471619129181, 219.66833090782166, 218.39710593223572]


In [63]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from torchsummary import summary

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GRU

In [2]:
class CSVDataset(Dataset):
    def __init__(self, path, chunksize, nb_samples, separator=','):
        self.path = path
        self.chunksize = chunksize
        self.len = int(nb_samples / self.chunksize)
        self.separator = separator
        self.all_letters = 'ACTG'

    def __getitem__(self, index):
        df = pd.read_csv(
                self.path,
                sep=self.separator,
                skiprows=index * self.chunksize + 1,  #skip header
                nrows=self.chunksize,
                header=None,
                usecols=[1,3])
        np_array = df.values
        one_hot_letters = self.lineToTensor(np_array[index,0])
        integers_labels = np.array(list(df.values[index,1]),dtype=int)
        x = one_hot_letters
        y = integers_labels
        return x,y
    
    def letterToTensor(self, letter):
        #change this for the K mers encoding
        tensor = torch.zeros(1, 4) #only 4 letters in vocabulary
        tensor[0][self.letterToIndex(letter)] = 1
        return tensor
    
    def letterToIndex(self, letter):
        return self.all_letters.find(letter)

    def __len__(self):
        return self.len
    
    def lineToTensor(self, line):
        tensor = torch.zeros(len(line), 1, 4)
        for li, letter in enumerate(line):
            tensor[li][0][self.letterToIndex(letter)] = 1
        return tensor

In [3]:
dataset = CSVDataset('Input_file.txt', chunksize=1, nb_samples=1,separator=';')
loader = DataLoader(dataset, batch_size=1, num_workers=2, shuffle=False)

In [4]:
class GRUNet(nn.Module):
    
    def __init__(self, hiddensize,inputsize):
        super(GRUNet, self).__init__()
        self.hidden_size = hiddensize
        self.input_size = inputsize
        self.gru = nn.GRU(input_size=inputsize,hidden_size=hiddensize,num_layers=1)
        self.linear = nn.Linear(hiddensize,2)
        

    def forward(self, input_x ,hidden):
        #print(input_x, hidden)
        output, hidden = self.gru(input_x, hidden)
        #print("OUT", output.shape)
        output = self.linear(output)
        #print("LINEAR", output)
        output = F.log_softmax(output, dim = 2)
       
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


In [5]:
losses = []
loss_function = nn.NLLLoss()
model = GRUNet(hiddensize=32, inputsize=4)
optimizer = optim.SGD(model.parameters(), lr=0.001)



In [17]:
last_prediction = []
for epoch in range(40):
    total_loss = 0
    for batch_idx, data in enumerate(loader):
        train = data[0]
        train_labels = data[1].view(-1)
        length_seq = 1000      
        for i in range(int(data[0].shape[1]/length_seq)):
            hidden = model.initHidden()
            portion_i = length_seq*i 
            portion_i1 = (i+1)*length_seq 
            X = train[:,portion_i:portion_i1,:,:]

            #y = torch.nn.functional.one_hot(data[1])
            #y = y.view(-1,2)
            
            y = train_labels[portion_i:portion_i1]
            
            model.zero_grad()
            #print(y)
            #print("First input", X[0,:,:,:].shape)
            #print("second input", hidden.shape)

            # Step 3. Run the forward pass, getting log probabilities over next
            # words
            output, hidden = model(X[0,:,:,:], hidden)
            output = output.view(-1,2)
            #print("output:",output)
            #print("label", y)
            loss = loss_function(output, y)
            if epoch == 39:
                last_prediction.append(output)


            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()
            print(i)

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
    losses.append(total_loss)
print(losses) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145


Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", 

KeyboardInterrupt: 

In [16]:
np.where(torch.cat(last_prediction).argmax(dim=1)==0)

(array([     0,      1,      2, ..., 158997, 158998, 158999]),)

In [9]:
last_prediction

[tensor([[-0.3934, -1.1233],
         [-0.3718, -1.1695],
         [-0.3637, -1.1878],
         ...,
         [-0.3774, -1.1573],
         [-0.3674, -1.1795],
         [-0.3623, -1.1909]], grad_fn=<ViewBackward>),
 tensor([[-0.4088, -1.0920],
         [-0.3785, -1.1548],
         [-0.3693, -1.1752],
         ...,
         [-0.3624, -1.1907],
         [-0.3539, -1.2106],
         [-0.3668, -1.1807]], grad_fn=<ViewBackward>),
 tensor([[-0.3928, -1.1244],
         [-0.3771, -1.1579],
         [-0.3809, -1.1496],
         ...,
         [-0.3501, -1.2196],
         [-0.3517, -1.2157],
         [-0.3484, -1.2236]], grad_fn=<ViewBackward>),
 tensor([[-0.4082, -1.0931],
         [-0.3847, -1.1415],
         [-0.3657, -1.1833],
         ...,
         [-0.3766, -1.1591],
         [-0.3616, -1.1926],
         [-0.3604, -1.1953]], grad_fn=<ViewBackward>),
 tensor([[-0.3951, -1.1196],
         [-0.3746, -1.1633],
         [-0.3680, -1.1780],
         ...,
         [-0.3696, -1.1744],
         [-0.3

In [219]:
for batch_idx, data in enumerate(loader):

    for seq in data[0]:
        print(seq.shape)

torch.Size([159624, 1, 4])


In [59]:
loss = nn.CrossEntropyLoss()
input = torch.ones(1, requires_grad=True)
target = torch.ones(1, dtype=torch.long)
print(input.size(),target.size())
output = loss(input, target)
print(input,target)
print("Loss: ", output)

torch.Size([1]) torch.Size([1])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [16]:
for i in range(int(10.6)):
    print(i)

0
1
2
3
4
5
6
7
8
9
